In [49]:
#%load_ext autoreload
%autoreload 2

import sqlite3
import psycopg2
import pandas as pd
import numpy as np
import math
import io
from DataBase_functions import *
from DataBase_functions_testing import *

#Google sheed API
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

<p style="font-size: 50px; text-align: center; color: #20a7e5;">Creating definitive data-frame</p>

<p style="font-size: 25px; color: #208ee5">Registro actividad todos</p>

In [50]:
# Email service account that need to share the google sheet: matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com
service_account_email = "" 

#Permisos que solicitará a la cuenta de google
scope = [
    'https://spreadsheets.google.com/feeds',        #Scope antiguo de Sheet API (v3)
    'https://www.googleapis.com/auth/spreadsheets', #Scope moderno de Sheet API (v4)
    'https://www.googleapis.com/auth/drive'         #Permisos para Google Drive
    ]

credentials = "credentials.json" #Credenciales del proyecto de Google Cloud

headers_definitive = ["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios", "version_date", "version", "petition_arq", "dev_master"]
try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1mUJAhezlVOj2TVWBs3loo8C91aDWzqaYjboDysidpO0" 
    #Registro actividad todos 2024 final: 1mUJAhezlVOj2TVWBs3loo8C91aDWzqaYjboDysidpO0
    #28.12.2024: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c 
    #23.01.2025: 1223iwU7YQTMEJ_zjRjWci_E6fDnwP09Bflt4b_iOJzo 
    #02.02.2025: 1UGvuP5F1-xRIA_oeyC2RYPBT0K5syiUiZyT75obap2I

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #Dataframe con los datos de todos los meses
    headers = ["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]
    df = pd.DataFrame(columns=headers) 
    meses = ["2024", "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre", "Enero_25"]
    for sheet in sheets:
        sheet_name = sheet.title
        print(sheet_name) 
        sheet.update(values=[headers], range_name='A1')
        if sheet_name in meses:
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=headers))
            df = pd.concat([df, df_aux], ignore_index=True)
        else: 
            print(sheet_name, "is not a month")
            pass
    df = df.drop_duplicates(subset='Código', keep='first')
    df = df[headers] #Únicamente me quedo con las columnas que me interesa
    print(df.shape, "\n", df.columns)

    #Dataframe validation
    df.rename(columns={"Validada": "validada", "Responsable": "responsable", "Gestor DDBB": "DDBB", "Geografía": "geography", "Petición": "petition_code", "Feature": "feature", "SDATOOL": "sdatool", "Código": "DQDP_code", "Gestor BBDD": "DDBB", "Horas": "duration_time", "Comentarios": "description", "Fecha de alta": "fecha_in", "Fecha de fin": "fecha_out"}, inplace=True)

    df["fecha_in"] = df["fecha_in"].astype(str)
    df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
    df["fecha_out"] = df["fecha_out"].astype(str)
    df["UUAA"] = df["UUAA"].astype(str)
    df["DQDP_code"] = df["DQDP_code"].astype(str)
    df["sdatool"] = df["sdatool"].astype(str)
    df["feature"] = df["feature"].astype(str)
    df["geography"] = df["geography"].astype(str)
    df["DDBB"] = df["DDBB"].astype(str)
    df["Ámbito"] = df["Ámbito"].astype(str)
    df["responsable"] = df["responsable"].astype(str)
    df["validada"] = df["validada"].astype(str)
    df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
    df["description"] = df["description"].astype(str)
    df["petition_code"] = df["petition_code"].astype(str)

    df["version_date"] = "Nan"
    df["version"] = "Nan"
    df["petition_arq"] = "Nan"
    df["dev_master"] = "Nan"

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

print(df.columns)

Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Enero_25', 'Responsables', 'None', 'Peticiones totales', 'All petitions', 'General']
2024 totales - Peticiones
2024 totales - Peticiones is not a month
2024 totales - Peticiones - Pro
2024 totales - Peticiones - Pro is not a month
2024
Gráficos Semanales - Procesos
Gráficos Semanales - Procesos is not a month
2024 - Procesos
2024 - Procesos is not a month
Abril
Mayo
Junio
Julio
Agosto
Septiembre
Octubre
Noviembre
Diciembre
Enero_25
Responsables
Responsables is not a month
None
None is not a month
Peticiones totales
Peticiones totales is not a month
All petitions
All petitions is not a month
General
General is not a month
(1802, 15) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 

<p style="font-size: 35px; color: #208ee5;">Peticiones globales Oracle, Elastic Search y Mongo DB</p>

<p style="font-size: 25px; color: #208cc5;;">Oracle Physics</p>

In [51]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1F4WCu-pofieOYEBnY8bo0t7-j9IoW5NRHs7BobUPmKo" 
    #23.01.2025: 1Ia7E6mfBtF6q9PcGwSA8Qa9OR4B8wTB_LjCjJnZo-70
    #28.12.2024: "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE" 
    #02.02.2025: 1F4WCu-pofieOYEBnY8bo0t7-j9IoW5NRHs7BobUPmKo


    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    headers = ["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Path", "Comentarios"]
    df_OG = pd.DataFrame(columns=headers)

    for sheet in sheets: 
        try: 
            print(sheet.title)
            sheet.update(values=[headers], range_name='A1') #Nombrando las columnas

            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=headers))
            df_OG = pd.concat([df_OG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_OG["DDBB"] = "Oracle Physics"
    df_OG["geography"] = "Global"

    print(df_OG.shape)

    df_OG["UUAA"] = df_OG["UUAA"].astype(str)
    df_OG["Petición Jira"] = df_OG["Petición Jira"].astype(str)
    df_OG["Versión Repo"] = df_OG["Versión Repo"].astype(str)
    df_OG["Petición ARQ"] = df_OG["Petición ARQ"].astype(str)
    df_OG["Fecha cierre"] = df_OG["Fecha cierre"].astype(str)
    df_OG["Comentarios"] = df_OG["Comentarios"].astype(str)
    df_OG["Path"] = df_OG["Path"].astype(str)
    df_OG["DDBB"] = df_OG["DDBB"].astype(str)
    df_OG["geography"] = df_OG["geography"].astype(str)

    df_OG.rename(columns={"Path": "path", "Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    df_OG = df_OG[["petition_code", "version", "petition_arq", "fecha_out", "description", "Path", "UUAA", "DDBB", "geography"]]
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

print(df_OG.columns)

Available sheeds: ['APIC', 'ATAU', 'B1DS', 'B43T', 'BJP9', 'CCUS', 'DAMS', 'DMTO', 'EGRP', 'EI4M', 'ESCD', 'FCRK', 'FZHI', 'GDEL', 'GHDC', 'GJSX', 'GSIN', 'GUAR', 'H9IY', 'HFPP', 'HLCY', 'HPGR', 'IMPT', 'J6G7', 'K5VI', 'KAPI', 'KARC', 'KBGE', 'KBTQ', 'KCCX', 'KCDO', 'KCMC', 'KCMS', 'KCNC', 'KCNS', 'KCOG', 'KCOL', 'KCSN', 'KCSP', 'KDCL', 'KDFH', 'KDMG', 'KECF', 'KERF', 'KESJ', 'KFGC', 'KFRE', 'KFUL', 'KGCC', 'KGCI', 'KGTP', 'KGWY', 'KIFV', 'KIVS', 'KLIM', 'KLNE', 'KMET', 'KMIC', 'KMOL', 'KMRK', 'KMYC', 'KNWD', 'KOMN', 'KONB', 'KPAG', 'KPAD', 'KPAY', 'KPDA', 'KPDR', 'KPFE', 'KPFM', 'KPIG', 'KPPS', 'KPST', 'KRBA', 'KRCE', 'KSAG', 'KSAN', 'KSDD', 'KSPH', 'KSRO', 'KUSU', 'KVCS', 'KXSG', 'KYSZ', 'KYOP', 'KYUB', 'MCRR', 'MFCG ', 'MK0W', 'MDDM', 'NI0C', 'O43T', 'PGVC', 'PIVU', 'PMSC', 'PMSV', 'P6ES', 'QWJR', 'QWPJ', 'RLR7', 'SWP3', 'S47B', 'UGX2', 'V34F', 'VAMO', 'W1BD', 'XBOB', 'ZGR7', 'XMBR', 'XR6Z', 'ZDNM', 'ZNXD']
APIC
ATAU
B1DS
B43T
BJP9
CCUS
DAMS
DMTO
EGRP
EI4M
ESCD
FCRK
FZHI
GDEL
GHDC
G

<p style="font-size: 25px; color: #208cc5;;">Elastic Search</p>

In [52]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "19hdomSp-b_cQB3oE-LcdNDS8X-Jurj4OsWJf0YyEUJ8" 
    #28.01.2024: 1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE
    #23.01.2025: 15Ak0o3xXZ22Xeg36pl10WMPQxQ_50x2IGG7ceUnP1Ec
    #02.02.2025: 19hdomSp-b_cQB3oE-LcdNDS8X-Jurj4OsWJf0YyEUJ8

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    headers = ["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]
    df_EG = pd.DataFrame(columns=headers)

    for sheet in sheets: 
        try: 
            print(sheet.title)
            sheet.update(values=[headers], range_name='A1') #Nombrando las columnas
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Path", "Comentarios"]))
            df_EG = pd.concat([df_EG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_EG["DDBB"] = "Oracle Physics"
    df_EG["geography"] = "Global"

    #df = df_OG.drop_duplicates(subset='Código', keep='first')
    print(df_EG.shape)

    df_EG["UUAA"] = df_EG["UUAA"].astype(str)
    df_EG["Petición Jira"] = df_EG["Petición Jira"].astype(str)
    df_EG["Versión Repo"] = df_EG["Versión Repo"].astype(str)
    df_EG["Petición ARQ"] = df_EG["Petición ARQ"].astype(str)
    df_EG["Fecha cierre"] = df_EG["Fecha cierre"].astype(str)
    df_EG["Comentarios"] = df_EG["Comentarios"].astype(str)
    df_EG["Path"] = df_EG["Path"].astype(str)
    df_EG["DDBB"] = df_EG["DDBB"].astype(str)
    df_EG["geography"] = df_EG["geography"].astype(str)

    df_EG.rename(columns={"Path": "path", "Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    df_EG = df_EG[["petition_code", "version", "petition_arq", "fecha_out", "description", "Path", "UUAA", "DDBB", "geography"]]
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

print(df_EG.columns)

Available sheeds: ['ATAU', 'F30V', 'EI4M', 'GHDC', 'GUAR', 'H9IY', 'IMPT', 'J6G7', 'JBSY', 'KAGE', 'KAPI', 'KBTQ', 'KCNS', 'KHR1', 'KIFV', 'KLNE', 'KMOL', 'KMYB', 'KREC', 'KTIN', 'KTRA', 'KTR2', 'KXSG', 'KYGU', 'KYOP', 'L1WI', 'NI0C', 'O43T', 'OPEI', 'PGVC', 'S47B', 'XBMR', 'XPI2', 'YM3X', 'ZNXD']
ATAU
F30V
EI4M
GHDC
GUAR
H9IY
IMPT
J6G7
JBSY
KAGE
KAPI
KBTQ
KCNS
KHR1
KIFV
KLNE
KMOL
KMYB
Quota exceeded. Retrying in 6 seconds...
KREC
Quota exceeded. Retrying in 6 seconds...
KTIN
KTRA
KTR2
KXSG
KYGU
KYOP
L1WI
NI0C
O43T
OPEI
PGVC
S47B
XBMR
XPI2
YM3X
ZNXD
(235, 10)
Error: "['Path'] not in index"
Index(['UUAA', 'petition_code', 'version', 'petition_arq', 'fecha_out',
       'description', 'path', '', 'DDBB', 'geography'],
      dtype='object')


<p style="font-size: 25px; color: #208cc5;">Mongo DB</p>

In [53]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1G4Lp7XaEnU9cOtIttsJaGwsH8aMUeX5W2ACDQ4RyY_c" 
    #28.12.2024: 1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs 
    #23.01.2025: 1HKsoAlC2Nm-YUI75-mqAQqBKyqPdc9TcKl2NckwpFew
    #02.02.2025: 1G4Lp7XaEnU9cOtIttsJaGwsH8aMUeX5W2ACDQ4RyY_c v1
    #02.02.2025: 1rhs6ccoRpGd1kf5PchmRFy-CVQ33IZA0drZoKdlKyZA v2

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    headers = ["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Path", "Comentarios"]
    df_MG = pd.DataFrame(columns=headers)

    for sheet in sheets: 
        try: 
            print(sheet.title)
            sheet.update(values=[headers], range_name='A1') #Nombrando las columnas
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Path", "Comentarios"]))
            df_MG = pd.concat([df_MG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_MG["DDBB"] = "Oracle Physics"
    df_MG["geography"] = "Global"
    
    print(df_MG.shape)

    df_MG["UUAA"] = df_MG["UUAA"].astype(str)
    df_MG["Petición Jira"] = df_MG["Petición Jira"].astype(str)
    df_MG["Versión Repo"] = df_MG["Versión Repo"].astype(str)
    df_MG["Petición ARQ"] = df_MG["Petición ARQ"].astype(str)
    df_MG["Fecha cierre"] = df_MG["Fecha cierre"].astype(str)
    df_MG["Comentarios"] = df_MG["Comentarios"].astype(str)
    df_MG["Path"] = df_MG["Path"].astype(str)
    df_MG["DDBB"] = df_MG["DDBB"].astype(str)
    df_MG["geography"] = df_MG["geography"].astype(str)

    df_MG.rename(columns={"Path": "path", "Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    df_MG = df_MG[["petition_code", "version", "petition_arq", "fecha_out", "description", "Path", "UUAA", "DDBB", "geography"]]
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

print(df_MG.columns)

Available sheeds: ['KSKR', 'KPDR', 'F30V', 'KAPI', 'H9IY', 'KPAD', 'CHM8', 'KCOG', 'KONL', 'KNWD', 'KIFV', 'KIFD', 'KPFM', 'KHR1', 'KARC', 'KATT', 'XMBR']
KSKR
KPDR
F30V
KAPI
H9IY
KPAD
CHM8
KCOG
KONL
KNWD
KIFV
KIFD
KPFM
KHR1
KARC
KATT
XMBR
(82, 9)
Error: "['Path'] not in index"
Index(['UUAA', 'petition_code', 'version', 'petition_arq', 'fecha_out', 'path',
       'description', 'DDBB', 'geography'],
      dtype='object')


<p style="font-size: 35px; color: #208ee5;">dqdp_portal CSV</p>

In [54]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

service = build('drive', 'v3', credentials=creds)

#Drive, folder id
folder_id = '1w2V9GaUCXD4vdjolQjIAEMujMOiqYwEP'

#list of archives
results = service.files().list(
    q = f"'{folder_id}' in parents", 
    pageSize=120, 
    fields = "nextPageToken, files(id, name)"
).execute()
files = results.get('files', [])

#List that can take all dataframe from csv files
dfs = []

# Itera sobre los archivos y lee los CSV
for file in files:
    if file['name'].endswith('.csv'): # Solo procesa archivos CSV
        file_id = file['id']
        # Descarga el archivo CSV a un archivo temporal
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO() # Usamos un archivo en memoria para no guardar el archivo
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()

        fh.seek(0) # Reiniciar el puntero al inicio del archivo
        try:
            df_aux = pd.read_csv(fh) # Lee el CSV con Pandas
            dfs.append(df_aux)
        except Exception as e:
            print(f"Error al leer el archivo {file['name']}: {e}")

# Combina todos los DataFrames en uno solo
df_combined = pd.concat(dfs, ignore_index=True)

In [55]:
df_combined.rename(columns={'Código': 'DQDP_code', 'Responsable': 'responsable', 'Fecha de alta': 'fecha_in', 'Petición': 'petition_code', 'Geografía': 'geography', 'Horas esfuerzo': 'duration_time', 'Gestor': 'DDBB', 'Fecha fin': 'fecha_out'}, inplace=True)
df_combined['description'] = df_combined['Descripción corta'] + '\n' + df_combined['Descripción']
df_combined = df_combined[['DQDP_code', 'UUAA', 'responsable', 'fecha_in', 'petition_code', 'geography', 'duration_time', 'DDBB', 'description']]

df_combined.columns

Index(['DQDP_code', 'UUAA', 'responsable', 'fecha_in', 'petition_code',
       'geography', 'duration_time', 'DDBB', 'description'],
      dtype='object')

<p style="font-size: 35px; color: #20cff5; text-align: center;">All in one Data Frame</p>

In [56]:
#Creating global data-frame
df_g = pd.DataFrame(columns=list(df.columns))
df_g = pd.concat([df_g, df_OG, df_EG, df_MG, df_combined], ignore_index=True)
print(df_g.columns)


columns_to_use = ["fecha_in", "fecha_out", "UUAA", "DQDP_code", "sdatool", "feature", "geography", "DDBB", "Ámbito", "responsable", "validada", "description", "version_date", "version", "petition_arq", "petition_code", "duration_time", "dev_master"]
dataframes = [df, df_g]
for column in columns_to_use: 
    for i in dataframes:
        i[column] = i[column].astype(str)

df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
df["dev_master"] = "None"

df_g["duration_time"] = pd.to_numeric(df_g["duration_time"], errors="coerce")
df_g["dev_master"] = "None"



#Poniendo "None" o np.nan donde corresponde
dataframes_using = [df, df_g]
for i in dataframes_using:
    for row in i.iterrows(): 
        for key, value in dict(row[1]).items():
            if value == "Nan" or value == None or value == "" or value == "nan" or value == "None" or value == "none" or value == "NONE" or value == np.nan: 
                if key == "fecha_in" or key == "fecha_out" or key == "Fecha incurrida" or key == "UUAA" or key == "DQDP_code" or key == "sdatool" or key == "feature" or key == "geography" or key == "DDBB" or key == "Ámbito" or key == "responsable" or key == "validada" or key == "description" or key == "version_date" or key == "version" or key == "petition_arq" or key == "petition_code" or key == "dev_master":
                    i.at[row[0], key] = "None"
                if key == "duration_time":
                    i.at[row[0], key] = np.nan
                else: 
                    pass



df_g = df_g[df_g["petition_code"] != "None"]
df = df[df["petition_code"] != "None"]
df = pd.concat([df, df_g], ignore_index=True)

agg_dict = {
    col: 'first' for col in df_g.columns if col not in ["description", "duration_time"]
}
agg_dict['description'] = lambda x: "\n".join(x.astype(str))
agg_dict["duration_time"] = 'sum'

#Data frame final a partir del campo 'poetition_code'
#df_g.drop_duplicates(subset=["petition_code"], keep="first", inplace=True) #Método 1
final_df = df.groupby("petition_code", as_index=False).agg(agg_dict) #Método 2

C:\Users\matia\AppData\Local\Temp\ipykernel_8500\32688967.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_g = pd.concat([df_g, df_OG, df_EG, df_MG, df_combined], ignore_index=True)


Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'duration_time', 'description',
       'version_date', 'version', 'petition_arq', 'dev_master', 'path', ''],
      dtype='object')


In [57]:
petitions = set(df_g["petition_code"]).union(set(df["petition_code"]))
print(len(petitions), final_df.columns)
petitions

4247 Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'version_date', 'version', 'petition_arq',
       'dev_master', 'path', '', 'description', 'duration_time'],
      dtype='object')


{'OTRSSD-4502',
 'OTRSSD-4014',
 'OTRSSD-15129',
 'OTRSSD-6051',
 'SGS-14080',
 'OTRSSD-5744',
 'OTRSSD-11436',
 'REQ000100138856',
 'REQ000100392878',
 'WO0000101369291',
 'OTRSSD-14690',
 'RRRG_20230516172956',
 'REQ000013251594',
 'OTRSSD-5082',
 'OTRSSD-5710',
 'OTRSSD-9643',
 'OTRSSD-8891',
 'OTRSSD-9724',
 'REQ000100693541',
 'OTRSSD-3855',
 'OTRSSD-4654',
 'OTRSSD-16316',
 'SGS-10486',
 'EBWR_20241204142729',
 'OTRSSD-8055',
 'OTRSSD-18444',
 'OTRSSD-10359',
 'OTRSSD-5484',
 'OTRSSD-6052',
 'TPCT_20240524104554',
 'OTRSSD-9277',
 'REQ000013117812',
 'REQ000100138871',
 'OTRSSD-12836',
 'OTRSSD-13609',
 'OTRSSD-13187',
 'FIGD_20240415130838',
 'OTRSSD-13211',
 'SGS-12983',
 'OTRSSD-16203',
 'OTRSSD-5197',
 'REQ000100585207',
 'OTRSSD-7547',
 'OTRSSD-3858',
 'OTRSSD-13477',
 'OTRSSD-5572',
 'SGS-14883',
 'OTRSSD-10133',
 'OTRSSD-16122',
 'REQ000013537639',
 'OTRSSD-4273',
 'OTRSSD-6755',
 'DSCZ_20220830132439',
 'OTRSSD-7625',
 'OTRSSD-15904',
 'OTRSSD-4663',
 'OTRSSD-8648',
 'REQ

In [58]:
#Poniendo "None" o None donde corresponda
for petition in petitions: 
    dic_aux = {i: [] for i in columns_to_use}

    for column in columns_to_use: 
        df_filtered = df[df["petition_code"] == petition]
        df_g_filtered = df_g[df_g["petition_code"] == petition]
        
        if not df_filtered.empty: 
            df_value = df_filtered[column].values[0]
        else: 
            df_value = "None" if column != "duration_time" else None

        if not df_g_filtered.empty: 
            df_g_value = df_g_filtered[column].values[0]
        else: 
            df_g_value = "None" if column != "duration_time" else None
        

        if df_value != "None" and df_value is not None: 
            dic_aux[column].append(df_value)
        elif df_g_value != "None" and df_g_value is not None: 
            dic_aux[column].append(df_g_value)
        else:
            dic_aux[column].append(df_value) #None o "None"

    df_aux = pd.DataFrame(dic_aux)
    final_df = pd.concat([final_df, df_aux], ignore_index=True)

agg_dict = {
col: 'first' for col in df_g.columns if col not in ["description", "duration_time"]
}
agg_dict['description'] = lambda x: "\n".join(x.astype(str))
agg_dict["duration_time"] = 'sum'
final_df = df.groupby("petition_code", as_index=False).agg(agg_dict)

In [59]:
#Contar los valores "None" o None
dfs = [df, df_g, final_df]
df_none_count = {i: 0 for i in list(dfs[2].columns)}
for row in dfs[2].iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "None" or value == None or (isinstance(value, float) and np.isnan(value)):
            df_none_count[key] += 1
        else: 
            pass
print(dfs[0].shape, dfs[1].shape, dfs[2].shape)
df_none_count


(7051, 21) (5250, 21) (4247, 21)


{'fecha_in': 698,
 'Fecha incurrida': 2521,
 'fecha_out': 1863,
 'UUAA': 1,
 'DQDP_code': 699,
 'sdatool': 2580,
 'feature': 2539,
 'petition_code': 0,
 'geography': 260,
 'DDBB': 3,
 'Ámbito': 2521,
 'responsable': 1244,
 'validada': 3383,
 'version_date': 4247,
 'version': 3552,
 'petition_arq': 3579,
 'dev_master': 4247,
 'path': 3110,
 '': 4207,
 'description': 1993,
 'duration_time': 0}

---
---

<p style="font-size: 45px; text-align: center; color: green;">Cleaning definitive dataframe</p>

In [60]:
print(final_df["geography"].unique())
#final_df.drop(columns=['', 'path'], inplace=True)
print(final_df.columns)

['Global' 'España' 'None' 'España ' 'CIB' 'España/CIB' 'España-CIB'
 'Holding' 'México' 'Perú' 'Colombia' 'Argentina']
Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'version_date', 'version', 'petition_arq',
       'dev_master', 'path', '', 'description', 'duration_time'],
      dtype='object')


In [61]:
final_df["DDBB"] = final_df["DDBB"].replace(["DB2 HOST"], "DB2 Host")
final_df["DDBB"] = final_df["DDBB"].replace(["Oracle Physics"], "Oracle Physics")
final_df["DDBB"] = final_df["DDBB"].replace(["ELASTICSEARCH", "ElasTICSEARCH", "ElaSTICSEARCH"], "Elastic Search")
final_df["DDBB"] = final_df["DDBB"].replace(["ORACLE R2", "oracle r2"], "Oracle R2")
final_df["DDBB"] = final_df["DDBB"].replace(["teradata", "TERADATA"], "Teradata")
final_df["DDBB"] = final_df["DDBB"].replace(["MongoDB", "MONGO DB", "MongoDB", "MONGODB"], "Mongo DB")
final_df["DDBB"] = final_df["DDBB"].replace(["POSTGRESS R2", "POSTGRESS Physics"], "PostgreSQL")
final_df["DDBB"] = final_df["DDBB"].replace(["España/CIB", "España-CIB"], "CIB")

final_df["geography"] = final_df["geography"].replace(["NETEZZA"], "Netezza")

In [62]:
final_df["fecha_in"] = final_df["fecha_in"].astype(str)
final_df["Fecha incurrida"] = final_df["Fecha incurrida"].astype(str)
final_df["fecha_out"] = final_df["fecha_out"].astype(str)
final_df["UUAA"] = final_df["UUAA"].astype(str)
final_df["DQDP_code"] = final_df["DQDP_code"].astype(str)
final_df["sdatool"] = final_df["sdatool"].astype(str)
final_df["feature"] = final_df["feature"].astype(str)
final_df["petition_code"] = final_df["petition_code"].astype(str)
final_df["geography"] = final_df["geography"].astype(str)
final_df["DDBB"] = final_df["DDBB"].astype(str)
final_df["responsable"] = final_df["responsable"].astype(str)
final_df["version"] = final_df["version"].astype(str)
final_df["petition_arq"] = final_df["petition_arq"].astype(str)
final_df["dev_master"] = final_df["dev_master"].astype(str)
final_df["path"] = final_df["dev_master"].astype(str)
final_df["description"] = final_df["description"].astype(str)
final_df["duration_time"] = pd.to_numeric(final_df["duration_time"], errors="coerce")

---

<p style="font-size: 25px; color: #00d17f;">Actualizar y guardar</p>

In [63]:
#Actualizar y guardar

# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

#Cargar credenciales
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

#Autorizar gspread con las credenciales
client = gspread.authorize(creds)

# ID de la hoja de cálculo 
srpeadsheet_id = "1biaKHw0fV5w5HBWsAto2Q-WCLkXe4bwc72hoS6BNDe8" #petitions file

# Open the Google Sheet file by ID
spreadsheet = client.open_by_key(srpeadsheet_id)

# Select worksheet
worksheet = spreadsheet.worksheet('All petitions')

set_with_dataframe(worksheet, final_df, include_index=False, include_column_header=True, resize=True)

###############################################################################################################
#Guardo el df en un excel en la carpeta actual
final_df.to_excel("petitions.xlsx", index=False, sheet_name="All petitions")

In [64]:
final_df

,fecha_in,Fecha incurrida,fecha_out,UUAA,DQDP_code,sdatool,feature,petition_code,geography,DDBB,...,responsable,validada,version_date,version,petition_arq,dev_master,path,,description,duration_time
0,None,None,13/09/2023,KMYC,None,None,None,,Global,Oracle Physics,...,None,None,None,23,DATASD-117636,None,None,None,cambiar el nombre de un índice\nNone,0.0
1,23/10/2024,11/11/2024,11/11/2024,KDHN,DQ-DP-007223,47764,ADMMAYOR-136,KDHN_20241022115246,España,DB2 Host,...,Pedro,None,None,None,None,None,None,None,ampliar un campo,3.0
2,None,None,03/02/2023,KTRA,None,None,None,OTRSSD-5682,Global,Oracle Physics,...,None,None,None,16,DATASD-62338,None,None,None,<Default>/3. Global/3.01 CS - Master/3.01.008 ...,0.0
3,19/12/2022,None,None,TVPV,DQ-DP-003399,None,None,-TVPV_20221212130816,España,DB2 Host,...,RAMON GARCIA IRANZO,None,None,None,None,None,None,None,None,0.0
4,28/02/2023,None,None,TVPV,DQ-DP-003823,None,None,A2027920,España,DB2 Host,...,JUAN ALFONSO BARRIOS PATIÑO,None,None,None,None,None,None,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4242,07/02/2025,None,None,KESJ,DQ-DP-007737,None,None,WO0000101440722,España,ORACLE Physics,...,MATIAS BLAQUIER PIROSANTO,None,None,None,None,None,None,None,None,1.0
4243,06/02/2025,None,None,FCRK,DQ-DP-007727,None,None,WO0000101440751,España,ORACLE Physics,...,JENNIFER DEL CARMEN ROJO CABARCAS,None,None,None,None,None,None,None,None,1.0
4244,07/02/2025,None,None,KGYM,DQ-DP-007741,None,None,WO0000101441611,España,ORACLE Physics,...,JENNIFER DEL CARMEN ROJO CABARCAS,None,None,None,None,None,None,None,None,0.0
4245,06/02/2025,None,None,SRII,DQ-DP-007731,None,None,WO0000101442051,España,ORACLE Physics,...,MATIAS BLAQUIER PIROSANTO,None,None,None,None,None,None,None,None,1.0


---
---

In [65]:
for row in final_df.iterrows():
    insert_data_testing(dict(row[1]))

<p style="font-size: 40px; color: grey;">Testing</p>

In [66]:
conn1 = sqlite3.connect("BBVA_testing.db")
cursor1 = conn1.cursor()
cursor1.execute("PRAGMA foreign_keys = ON") #In sqlite3 foreign keys are disabled by default

#######
cursor1.execute("SELECT UUAA FROM UUAA")
all_uuaa = [x[0] for x in cursor1.fetchall()]

#######
conn1.commit()
cursor1.close()
conn1.close()

In [67]:
'''
conn3 =  psycopg2.connect(
        dbname = "pwd_control_plnk", 
        user = "matublaq",
        password = "SF19KOpSPMl8Ru51ONQ33AHOf0RuZnne", 
        host = "dpg-ctevf3t6l47c73b4jadg-a.oregon-postgres.render.com",
        port = "5432"
)   
cursor3 = conn3.cursor()

##########


##########
conn3.commit()
cursor3.close()
conn3.close()
'''

'\nconn3 =  psycopg2.connect(\n        dbname = "pwd_control_plnk", \n        user = "matublaq",\n        password = "SF19KOpSPMl8Ru51ONQ33AHOf0RuZnne", \n        host = "dpg-ctevf3t6l47c73b4jadg-a.oregon-postgres.render.com",\n        port = "5432"\n)   \ncursor3 = conn3.cursor()\n\n##########\n\n\n##########\nconn3.commit()\ncursor3.close()\nconn3.close()\n'